In [7]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time
from tqdm import tqdm
from scipy.special import softmax
import torch.utils.data as tdata
import matplotlib.pyplot as plt 
import torch.backends.cudnn as cudnn
import random

In [8]:
import os
import shutil
from google.colab import drive
drive.mount('/content/drive')
checkpoints = '/content/drive/My Drive/Colab Notebooks/cse559a/'
if not os.path.exists(checkpoints):
  os.makedirs(checkpoints)
  
if not os.path.exists('imagenet_val'):
  if not os.path.exists(checkpoints + 'imagenet_val.tar'):
    print("Downloading archive...")
    os.chdir(checkpoints)
    !wget -nv -O imagenet_val.tar -L https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar
    os.chdir('/content/')
  print("Copying to local runtime...")
  shutil.copy(checkpoints + 'imagenet_val.tar', './imagenet_val.tar')
  print("Uncompressing...")
  !mkdir imagenet_val
  !tar -xf imagenet_val.tar -C ./imagenet_val/
  !rm imagenet_val.tar
  os.chdir('./imagenet_val') 
  !wget -qO- https://raw.githubusercontent.com/soumith/imagenetloader.torch/master/valprep.sh | bash
  os.chdir('/content/')
print("Data ready!")

if not os.path.exists('exp_v0.py'):
  shutil.copy(checkpoints + 'exp_v0.py', './exp_v0.py')
  print("File imported")
from exp_v0 import *

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Data ready!


In [9]:
class ConformalModel(nn.Module):
    def __init__(self, model, calib_loader, alpha, kreg=None, lamda=None, randomized=True, batch_size=32):
        super(ConformalModel, self).__init__()
        self.model = model 
        self.alpha = alpha
        self.T = torch.Tensor([1.3]) #initialize (1.3 is usually a good value)
        self.T, calib_logits = platt(self, calib_loader)
        self.randomized=randomized
        self.num_classes = len(calib_loader.dataset.dataset.classes)
        self.penalties = np.zeros((1, self.num_classes))
        self.penalties[:, kreg:] += lamda 

        calib_loader = tdata.DataLoader(calib_logits, batch_size = batch_size, shuffle=False, pin_memory=True)

        self.Qhat = conformal_calibration_logits(self, calib_loader)

    def forward(self, *args, randomized=None,  **kwargs):
        if randomized == None:
            randomized = self.randomized
        logits = self.model(*args, **kwargs)
        
        with torch.no_grad():
            logits_numpy = logits.detach().cpu().numpy()
            scores = softmax(logits_numpy/self.T.item(), axis=1)

            I, ordered, cumsum = sort_sum(scores)

            S = gcq(scores, self.Qhat, I=I, ordered=ordered, cumsum=cumsum, penalties=self.penalties, randomized=randomized)

        return logits, S


# Temperature scaling
def platt(cmodel, calib_loader, max_iters=10, lr=0.01, epsilon=0.01):
    print("Begin Platt scaling.")
    # Save logits so don't need to double compute them
    logits_dataset = get_logits_targets(cmodel.model, calib_loader)
    logits_loader = torch.utils.data.DataLoader(logits_dataset, batch_size = calib_loader.batch_size, shuffle=False, pin_memory=True)

    nll_criterion = nn.CrossEntropyLoss().cuda()

    T = nn.Parameter(torch.Tensor([1.3]).cuda())

    optimizer = optim.SGD([T], lr=lr)
    for iter in range(max_iters):
        T_old = T.item()
        for x, targets in logits_loader:
            optimizer.zero_grad()
            x = x.cuda()
            x.requires_grad = True
            out = x/T
            loss = nll_criterion(out, targets.long().cuda())
            loss.backward()
            optimizer.step()
        if abs(T_old - T.item()) < epsilon:
            break 

    print(f"Optimal T={T.item()}")
    return T, logits_dataset 

In [10]:
transform = transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                      std= [0.229, 0.224, 0.225])
            ])

modelName = 'ResNet152'
cudnn.benchmark = True
batch_size = 128
KREG_VAL = 5
LAMBDA_VAL = 0.01
NUM_TRIALS = 10

df_res = pd.DataFrame(columns=['calib_size','top1_coverage','top5_coverage','raps_coverage', 'raps_size'])

for i in range(NUM_TRIALS):
  for calib_size in [500, 2000, 5000, 10000, 20000]:
    cal_data, val_data, _ = torch.utils.data.random_split(torchvision.datasets.ImageFolder('./imagenet_val/', transform), [calib_size, 20000, 30000-calib_size])
    calib_loader = torch.utils.data.DataLoader(cal_data, batch_size=batch_size, shuffle=True, pin_memory=True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True, pin_memory=True)
    model = create_model(modelName)
    print(f"Size of the calibration dataset: {calib_size}")
    cmodel = ConformalModel(model, calib_loader, alpha=0.1, kreg=KREG_VAL, lamda=LAMBDA_VAL)
    top1_coverage, top5_coverage, raps_coverage, raps_size = validate(val_loader, cmodel, print_bool=True)
    temp_res = {'calib_size': calib_size,'top1_coverage': top1_coverage,'top5_coverage': top5_coverage,'raps_coverage': raps_coverage, 'raps_size': raps_size}
    df_res = df_res.append(temp_res, ignore_index = True)

Size of the calibration dataset: 500
Begin Platt scaling.
Computing logits for model (only happens once).


100%|██████████| 4/4 [00:05<00:00,  1.27s/it]


Optimal T=1.296091079711914
N: 20000 | Time: 0.346 (1.270) | Cvg@1: 0.750 (0.783) | Cvg@5: 1.000 (0.941) | Cvg@RAPS: 0.812 (0.886) | Size@RAPS: 3.625 (4.105)


<ipython-input-10-524952173a65>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_res = df_res.append(temp_res, ignore_index = True)


Size of the calibration dataset: 2000
Begin Platt scaling.
Computing logits for model (only happens once).


...

<ipython-input-10-524952173a65>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_res = df_res.append(temp_res, ignore_index = True)


Size of the calibration dataset: 20000
Begin Platt scaling.
Computing logits for model (only happens once).


100%|██████████| 157/157 [03:17<00:00,  1.26s/it]


Optimal T=1.193920612335205
N: 20000 | Time: 0.300 (1.271) | Cvg@1: 0.875 (0.781) | Cvg@5: 0.969 (0.939) | Cvg@RAPS: 0.875 (0.899) | Size@RAPS: 3.594 (3.760)


<ipython-input-10-524952173a65>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_res = df_res.append(temp_res, ignore_index = True)


In [11]:
print(df_res)

    calib_size  top1_coverage  top5_coverage  raps_coverage  raps_size
0        500.0        0.78335        0.94070        0.88630    4.10520
1       2000.0        0.78370        0.94340        0.89210    3.70490
2       5000.0        0.78200        0.94125        0.91280    4.33665
3      10000.0        0.78490        0.94005        0.90355    4.03925
4      20000.0        0.77915        0.93975        0.89835    3.85250
5        500.0        0.78490        0.94210        0.92890    5.02480
6       2000.0        0.77975        0.94005        0.89330    4.05325
7       5000.0        0.77900        0.94005        0.89705    3.94760
8      10000.0        0.78290        0.93880        0.89155    3.66110
9      20000.0        0.78505        0.94140        0.90085    3.86545
10       500.0        0.77835        0.93980        0.88155    3.96495
11      2000.0        0.78380        0.93920        0.90995    4.50045
12      5000.0        0.78625        0.94200        0.90200    3.91860
13    

In [12]:
mean_res = df_res.groupby('calib_size')[['top1_coverage', 'top5_coverage','raps_coverage', 'raps_size']].mean()
print(mean_res)

            top1_coverage  top5_coverage  raps_coverage  raps_size
calib_size                                                        
500.0            0.782000       0.940270       0.899500   4.368020
2000.0           0.782285       0.940550       0.896815   4.016885
5000.0           0.783390       0.940905       0.900435   3.960175
10000.0          0.783155       0.939965       0.898710   3.898550
20000.0          0.782125       0.939780       0.899435   3.829120
